## Socioeconomic data and TOC entitlements
* Entitlements assigned to census tracts
* Which census tracts (what income levels or median household income) have seen TOC entitlements?

In [4]:
import intake
from IPython.display import display, Markdown
import numpy as np
import pandas as pd
import geopandas as gpd
import utils
import boto3

In [5]:
catalog = intake.open_catalog("../catalogs/*.yml")

s3 = boto3.client('s3')
bucket_name = 'city-planning-entitlements'

In [8]:
tracts = catalog.census_tracts.read()

keep_cols = ['GEOID10', 'geometry', 'HD01_VD01']

tracts = tracts[keep_cols]

tracts.rename(columns = {'GEOID10':'GEOID', 'HD01_VD01': 'pop'}, inplace = True)

/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:4238: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(**kwargs)


In [20]:
crosswalk_parcels_tracts = gpd.sjoin(parcels[['AIN', 'geometry']], 
                                     tracts[['GEOID', 'pop', 'geometry']], 
                                     how = 'inner', op = 'intersects').drop(columns = 'index_right')

In [11]:
parcels = gpd.read_file(
    f"s3://{bucket_name}/gis/intermediate/toc_eligible_parcels_with_entitlements.geojson"
)

In [15]:
# Join parcels to census tracts
join1 = gpd.sjoin(parcels, tracts, how = 'inner', op = 'intersects').drop(columns = 'index_right')

In [ ]:
toc_tiers = utils.reconstruct_toc_tiers_file()
